# 𝐃𝐚𝐭 𝐒𝐜𝐢𝐞𝐧𝐜𝐞 𝐏𝐫𝐨𝐣𝐞𝐜𝐭: Marketing Ad challange


$\huge{𝐀𝐮𝐭𝐡𝐨𝐫: 𝐏𝐚𝐫𝐥𝐚𝐝 \ 𝐃𝐡𝐮𝐧𝐠𝐚𝐧𝐚}$

#        This project will predict who will most likely click on the AD.

Let's assume that you have a marketing campaign for which we spend 1000USD for acquiring one potential customer. For each customer that we target with our ad campaign and that clicks on the ad, we get an overall profit of 100USD. However, if we target a customer that ends up not clicking on the ad, we incur a net loss of 1000USD. Therefore, we can conclude that for each customer that was not targeted by the campaign and who clicks on the ad, we get an overall profit of 1100USD. How would you approach this problem?

In [ ]:
# Importing librairies
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, make_scorer, fbeta_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
%matplotlib inline 




In [ ]:
#Loading Data
data = pd.read_csv("advertising_dsdj.csv")
#verifying loaded data
data.head()#printing five line from the top

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().any()#looking for the missing value in the data

In [ ]:
#checking null value in the data set
data.isnull().sum()

In [ ]:
data = data.dropna()#removing null value from the data
data.isnull().any()# checking is their still null value in the data

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

In [ ]:
data.duplicated().any()# checking for any duplicates data in the dataset

In [ ]:
print("Number of duplicated records in the data sets = ",data.duplicated().sum())

In [ ]:
#printing duplicate data
data[data.duplicated()]

In [ ]:
data = data.drop_duplicates()#removing duplicate data from the data set
data.duplicated().any()

In [ ]:
data.shape

In [ ]:
data.info()

# Exploratory Data Analysis (EDA)

In [ ]:
#extract Descriptive statistics
data.describe()

We can observe something unusual in the age column wher the minimum age is -25 years and maximum age is too high 999 years.

In [ ]:
# sorting the age 
sorted_age = sorted(data["Age"])
index = [] #initaliging empty index
for i in range(len(sorted_age)):
    index.append(i)#adding the sorted age into the empty index
    
#Plotting sorted age against their index
x = index
y = sorted_age

plt.figure(figsize = (10, 5))# figure size
plt.scatter(x, y, s = 10)
plt.axhline(y = 0, linestyle = "--", color = "r")# adding horizontal line y = 0
plt.axhline(y = 100, linestyle = "--", color = "r")# adding horizontal line y = 100
#plt.show()

In [ ]:
data[(data['Age'] > 100) | (data['Age'] < 18)]

In [ ]:
# Removing the extreme age values
data = data[(data['Age'] >= 18) & (data['Age'] < 100)]
data.head()

In [ ]:
data.shape

To be able to work with thwe feature "Timestamp", we will transmorm it to datetime, a format that will allow calculation on it

In [ ]:
data["Timestamp"] = pd.to_datetime(data["Timestamp"])
data.head()

Check out if the daliy time spent on site is smaller or equal to the daily internet use. For this we creat a variable 
"Delta". Dalta = Daily Internet Usage - Daily Time Spent on Site

In [ ]:
# Let's check out if the 'Daily TIme Spend on Site' 
# is actually smaller or equal to the Daily Internet Usage'

data['delta'] = data['Daily Internet Usage'] - data['Daily Time Spent on Site']
data.head()
#sum(data['delta'] < 0)

In [ ]:
invalid_dalta = data['delta']<0
invalid_dalta = data[invalid_dalta]
invalid_dalta



In [ ]:
# Removing rows with a delta smaller than zero
data = data[data['delta'] >= 0]

# I'll remove the column that I just created, but you could definitely keep it :-) 
data = data.drop('delta', axis=1)

data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

# Descriptive Figures

cross correlation  plot

In [ ]:
# Plot heatmap of all data with correlation coefficients visible
data_corr = data.corr()
plt.subplots(figsize=(10,5))
sns.heatmap(data_corr, cmap='YlGnBu', linewidth=.005, annot=True)

# Distribution Plots With Respect To Our Target Feature

In [ ]:
# Subsetting by clicks
click = data["Clicked on Ad"] == 1
no_click = data["Clicked on Ad"] == 0

# Creating the plots
features = ["Daily Internet Usage", "Daily Time Spent on Site", "Area Income", "Age"]
labels = ["Click", "No Click"]

def creating_plots(features, labels):
    plt.figure(figsize=(15,7))
    for feature in features:
        plt.subplot(2, 2, features.index(feature)+1)
        sns.distplot(data[feature][click], label=labels[0], color="r")
        sns.distplot(data[feature][no_click], label=labels[1], color="b")
        plt.axvline(data[feature][click].mean(), linestyle = '--', color="r")
        plt.axvline(data[feature][no_click].mean(), linestyle = '--', color="b")
        plt.legend()

creating_plots(features, labels)

In [ ]:
table_country = pd.crosstab(index=data["Country"],     
                      columns="count")  
table_country= table_country= table_country.sort_values(["count"], ascending=False)
table_country.head()


In [ ]:
print("Max. user", table_country.max())
print("Min. user", table_country.min())

We can see that the users are fairly spread out acroos the globe; the the maximum users from 
one country reaches only 9, and that are 237 different countries to consider. Therefore it 
is not appropriate 
to look at the city distribution.

In [ ]:
# Functions that we will use later on
def data_info(data):
    return data.info()

def creating_boxplots(size, target, features, data):
    plt.figure(figsize=size)
    for i in range(len(numerical_vars)):
        plt.subplot(5, 3, i+1)
        sns.boxplot(x=target, y=features[i], data=data )
        
def creating_distplot(size, data, features):
    plt.figure(figsize=size)
    for i in range(len(features)):
        plt.subplot(5, 3, i+1)
        sns.distplot(data[features[i]])
        
def crossCorrelation(data):
    corr = data.corr()
    plt.figure(figsize=(15,8))
    sns.heatmap(corr, 
          xticklabels=corr.columns.values,
          yticklabels=corr.columns.values)
    print(corr)

    
num_vars = data.select_dtypes(exclude=['object', 'datetime64']).columns

# Creating distribution plots
creating_distplot((16,18), data, num_vars)

From the overall distributions, we can see interesting 
bimodal distributions for the "daily internet usage", and "daily 
time spent on site" which could suggest the presence of more than one group. 
Furthermore, the "Age" and "area income" are little bit skewed to the right and left respectively which could
potentially bias
our model later on. In a subsequent step, we have to apply a log transformation at least on the age feature.




# Log Transform

Before performing machine learning, we will transform some features to avoid
introducing some bias due to the overall distribution of the latter. As mentioned previously,
we have some skewed distributions and they can make our models underperformed. To potentially 
solve this issue, we will apply a logarithmic transformation to reduce the effect of outliers 
and reduce the distribution's range.

From our previous EDA, we observe that "Age" was right skewed; 
therefore this transformation will be applied to this feature.



In [ ]:
data["log_age"] = np.log(data["Age"])
data.head()

# Removing Some Features

In [ ]:
New_Data= data.drop(["Ad Topic Line", "Timestamp", "City"], axis=1)
New_Data.head()


# Normalization Of Numerical Features

In [ ]:
# Defining categorical, numerical, and datetime variables that we will use later
categorical_vars = ["Ad Topic Line", "City", "Country"]
numerical_vars = ["Daily Time Spent on Site", "Area Income", "Daily Internet Usage", "Male", "log_age"]
datetime_vars = "Timestamp"
target = "Clicked on Ad"


In [ ]:
# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)


In [ ]:
# Applying the MinMax transformation of the numerical variables
data = pd.DataFrame(data = New_Data)
data[numerical_vars] = scaler.fit_transform(data[numerical_vars])
data.head()

# Processing Categorical Feature: One Hot-Encoding

In [ ]:
# First, here is a one-hot encoding function to properly encode the- 
#categorical features for machine learning algorithm. 
# This function is fed with the entire dataset
final_data = pd.get_dummies(data)
final_data.head()